In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
import os

from dotenv import load_dotenv

load_dotenv()

# Set the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


In [2]:
# Create an instance of the WikipediaAPIWrapper class with the top result and a maximum of 200 characters for each document
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

# Create an instance of the WikipediaQueryRun class using the WikipediaAPIWrapper instance
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

# Create a WebBaseLoader instance with the URL of the website to be loaded
loader = WebBaseLoader("https://orange-forest-047f26a10.4.azurestaticapps.net/")

# Load the documents from the specified website
docs = loader.load()

# Split the loaded documents into chunks of 1000 characters with an overlap of 200 characters
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Create a FAISS database from the split documents using OpenAIEmbeddings
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())

# Create a retriever from the FAISS database
retriever = vectordb.as_retriever()

# Display the retriever
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x15cbb6d50>)

In [3]:
# Tools for the agent to use
# wikipedia tool for the agent to use to search for information
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)


#  tool for the agent to use to search for information about App
retriever_tool=create_retriever_tool(retriever,"App_search",
                                        "Search for information about the application. For any questions related to the application, you must use this tool!")


# arxiv tool for the agent to use to search for information 
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)


# Tool kit for the agent to use
tools =[wiki,retriever_tool,arxiv]

In [5]:
# create an llm instance and prompt template
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Default prompt can be changed to any prompt
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [6]:
# Create an agent with the tools and the llm instance
agent = create_openai_tools_agent(llm,tools,prompt)

# agent executor to run the agent
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [12]:
agent_executor.invoke({"input":"What can I do as a recruiter on this platform?"})



> Entering new AgentExecutor chain...

Invoking: `App_search` with `{'query': 'recruiter on this platform'}`


NeunetYou need to enable JavaScript to run this app.As a recruiter on this platform, you can use Neunet to streamline your recruitment process. Neunet provides tools and features to help recruiters find and connect with top talent efficiently. You can post job listings, search for candidates, manage applications, and communicate with potential hires through the platform. Additionally, Neunet offers AI-powered tools to assist in candidate screening and selection.

> Finished chain.


{'input': 'What can I do as a recruiter on this platform?',
 'output': 'As a recruiter on this platform, you can use Neunet to streamline your recruitment process. Neunet provides tools and features to help recruiters find and connect with top talent efficiently. You can post job listings, search for candidates, manage applications, and communicate with potential hires through the platform. Additionally, Neunet offers AI-powered tools to assist in candidate screening and selection.'}